# Asignmnet 8: Rasters

## Group 3: Mauricio Flores Jiménez, Claudia Córdova Yamauchi, Fátima Trujillo

The assigment consists of extracting 15 variables from the provided rasters for all department polygons in Peru, which contain the percentage of district area covered by a specific Morphological Settlement Zone. After obtaining these values, the requirement consists of generating choropleth maps for each of the 15 variables using Folium. As a final output, the code generates an HTML the map layers with a Layer Control.

The assigment is developed on Google Colab, instead of the traditional use of Jupyter Notebook, due to issues on RAM space. This also means that, in order for the code to work properly, the folders should be aligned with the members' drive folders.


## 1. Setting the environment

The first part of the process involves setting the environment, which means installing and calling the necesary libraries.

### 1.1 Libraries and packages

In [1]:
# As a first step, we install the libraries needed to work with rasters

!pip install rasterio
!pip install rasterstats

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 5.7 MB/s eta 0:00:00


In [2]:
#Then, we import libraries and packages

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling
from rasterio.mask import mask
from rasterio.merge import merge
from rasterstats import zonal_stats
from shapely.ops import transform
import pyproj
import glob
import os
import folium
from folium import Choropleth, LayerControl
import branca.colormap as cm

The libraries and packages called are useful for geospatial analysis, raster data manipulation, and creating interactive maps.

In general, they serve various purposes in geospatial data analysis and visualization. NumPy and pandas are used for general data manipulation, with NumPy handling numerical computations and arrays, while pandas manages structured data in DataFrames. 

For geospatial data, GeoPandas extends pandas for working with geographic information, such as shapefiles, and shapely is employed for geometric operations like transforming shapes.

Rasterio is central for working with raster datasets (like satellite images), allowing us to read, write, manipulate, and mask raster data, and rasterstats computes summary statistics for spatial intersections between vector and raster data. pyproj is used for reprojecting geographic coordinates between different coordinate systems.

For mapping, folium is used to create interactive maps, with Choropleth and LayerControl from folium allowing the creation of layered choropleth maps. branca.colormap supports customizing color scales on the maps. Finally, glob and os provide utilities for file management and system-level operations, like searching for files or interacting with the operating system.

### 1.2 Drive access

This part of the code includes setting up the access that will be useful for our work on Colab

In [3]:
#Interacting with Google Drive when using Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Working with shapefiles and rasters

In this part of the code, we read and process the data given in the shapefiles and the rasters. In order to do that, we set the shapefiles and rasters in the same crs. Then, as the shapefiles are at a district level, we loop over all the districts found on the raster that match the shapefile (i.e. all the districts of Peru) and we read the data contained in the rasters.

### 2.1 Shapefile

In this first part, we focus exclusively on the shapefile. This means reading it, applying the crs transformation to all the geomteries on it, and adjusting it to make it easier to work with.

In [4]:
# Loading and reading the shapefile
shapefile_path = '/content/drive/MyDrive/ColabNotebooks/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp'
gdf = gpd.read_file(shapefile_path)

In [5]:
#Transforming the shapefile to the same crs as the raster
import pyproj
transformer = pyproj.Transformer.from_crs('epsg:4326', 'esri:54009', always_xy=True)

# Defining a function to apply the transformation
def apply_transform(geom):
    return transform(transformer.transform, geom)

# Applying the transformation to the geometries
gdf['geometry'] = gdf['geometry'].apply(apply_transform)

In [12]:
#Defining the geometry
gdf['area_geo'] = gdf.geometry.area

#Setting column names
gdf.sort_values("area_geo", ascending = False ) \
        .loc[:, ['NOMBDEP', 'NOMBPROV', 'NOMBDIST', 'CODIGO', 'area_geo']]

<ipython-input-12-add855b324d3>:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area_geo'] = gdf.geometry.area


,NOMBDEP,NOMBPROV,NOMBDIST,CODIGO,area_geo
1725,LORETO,MAYNAS,NAPO,160107,2.433384e+10
1782,MADRE DE DIOS,TAMBOPATA,TAMBOPATA,170101,2.084185e+10
1733,LORETO,LORETO,TIGRE,160303,2.026304e+10
1621,UCAYALI,PURUS,PURUS,250401,1.853812e+10
1120,LORETO,PUTUMAYO,YAGUAS,160804,1.808389e+10
...,...,...,...,...,...
486,LIMA,LIMA,LINCE,150116,2.762747e+06
1760,LA LIBERTAD,TRUJILLO,FLORENCIA DE MORA,130103,2.523665e+06
148,PIURA,SULLANA,BELLAVISTA,200602,2.174532e+06
653,CALLAO,CALLAO,CARMEN DE LA LEGUA REYNOSO,070103,1.924534e+06


### 2.2 Rasters

For this part, we load the corresponding rasters, by creating a loop that read all the .tif files on our folder. We go district by district when reading it in order to keep free space on our RAM memory.

In [11]:
# Loading the first raster file
raster_path = 'drive/MyDrive/ColabNotebooks/Tiles/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif'

In [14]:
# Directory where the .tif files are located
directory_path = 'drive/MyDrive/ColabNotebooks/Tiles'

# Getting the full paths of all .tif files in the directory
tif_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.tif')]

# Creating an empty dataframe for all our results
save_docs = []

# Loop to get the stats of the rasters

for tif_file in tif_files:
    #Printing the file to know where we are located while the code is running
    print(tif_file)
    # Zonal statistics
    stats = zonal_stats(gdf, raster_path, stats="unique", categorical=True)
    # Adding the results to the GeoDataFrame
    df1 = pd.DataFrame(stats)
    df1['UBIGEO'] = gdf['UBIGEO']
    save_docs.append(df1)

drive/MyDrive/ColabNotebooks/Tiles/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C11.tif
drive/MyDrive/ColabNotebooks/Tiles/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif
drive/MyDrive/ColabNotebooks/Tiles/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C11.tif
drive/MyDrive/ColabNotebooks/Tiles/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C12.tif
drive/MyDrive/ColabNotebooks/Tiles/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif
drive/MyDrive/ColabNotebooks/Tiles/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C12.tif
drive/MyDrive/ColabNotebooks/Tiles/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C12.tif


### 2.2 Merging the shapefile and the rasters

In this part, we work over our results after reading the rasters, and we adjust our results to merge them with the shapefiles, all at a region level.

Then, in order to get the right stats from each region, we adjust the columns in the order wanted.

In [15]:
# Replacing missing values and grouping results by UBIGEO
df_ubigeo = pd.concat(save_docs).replace(np.nan, 0 ) \
    .groupby('UBIGEO').sum().reset_index()

In [17]:
#Meging data with the shapefile and grouping by region, our objetc of study
dta_final_depa = gdf[['UBIGEO', 'CCDD', 'NOMBDEP']] \
    .merge(df_ubigeo, on = ['UBIGEO'], how = 'left') \
    .drop(['UBIGEO'], axis = 1 ) \
    .groupby(['CCDD', 'NOMBDEP']).sum().reset_index()

In [18]:
#Showing the columns current state
dta_final_depa.columns

Index(['CCDD', 'NOMBDEP', 'unique', 0, 1, 2, 3, 4, 5, 11, 12, 13, 21, 22, 23,
       14, 24],
      dtype='object')

In [19]:
# Setting the column order wanted

#Creating a list with the order wanted
column_order = ['CCDD', 'NOMBDEP', 'unique', 0, 1, 2, 3, 4, 5, 11, 12, 13, 14, 21, 22, 23, 24]

# Reorderin the columns
dta_final_depa = dta_final_depa[column_order]

In [20]:
#Showing our data final result
dta_final_depa

,CCDD,NOMBDEP,unique,0,1,2,3,4,5,11,12,13,14,21,22,23,24
0,01,AMAZONAS,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,02,ANCASH,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,03,APURIMAC,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,04,AREQUIPA,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,05,AYACUCHO,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,06,CAJAMARCA,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,07,CALLAO,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,08,CUSCO,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,09,HUANCAVELICA,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,HUANUCO,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Saving the dataframe in our folder
file_path = '/content/drive/MyDrive/ColabNotebooks/dta_final_depa.csv'
dta_final_depa.to_csv(file_path, index=False)

## 3. Creating and saving the map

This part of the code creates a choropleth map of Peru, visualizing data for the raster variables, using Folium. It simplifies the geometries of the shapefile to reduce the file size, and then generates a base map centered on Peru. 

For each variable in a predefined list, it iterates and creates a separate choropleth layer with a specific color palette. Additionally, a layer control is added to allow toggling between the different variable layers.

As a final step, it creates and saves an HTML file with the results.

### 3.1. Generating the map

In [ ]:
# Simplifying the geometries to reduce the final file size (adjusting the tolerance parameter as necessary)
gdf['geometry'] = gdf['geometry'].simplify(tolerance=0.01, preserve_topology=True)

# Creating a base map centered on Peru
m = folium.Map(location=[-9.19, -75.0152], zoom_start=5)

# Iterating over each variable (from the relevant columns)
for col in ['0', '1', '2', '3', '4', '5', '11', '12', '13', '14', '21', '22', '23', '24']:
    # Creating a choropleth map for each variable with a discrete color palette
    choropleth = folium.Choropleth(
        geo_data=gdf[['NOMBDEP', 'geometry']],  # Including only geometry and department name
        name=f'Variable {col}',  # Naming the layer
        data=dta_final_depa,  # DataFrame with the final data
        columns=['NOMBDEP', col],  # Using the columns, department name and the variable
        key_on='feature.properties.NOMBDEP',  # Linking shapefile properties with the variable
        fill_color='YlGnBu',  # Setting the color palette
        fill_opacity=0.6,  # Reducing opacity to save space
        line_opacity=0.1,  # Reducing the border opacity to minimize size
        legend_name=f'Variable {col}',  # Adding the legend for the layer
        nan_fill_color="transparent",  # Setting color for missing values, helping to reduce size
        bins=8,  # Reducing the number of bins to make the legend and color palette more efficient
    )
    choropleth.add_to(m)

# Adding layer control
folium.LayerControl().add_to(m)

KeyError: '0'

### 3.2 Saving as HTML

In [ ]:
# Saving the map in an HTML file
output_path = "/content/drive/MyDrive/ColabNotebooks/group_3_ass_8_2024_2.html"
m.save(output_path)

#Printing to ensure the result
print(f'Map saved as {output_path}')

Mapa guardado como /content/drive/MyDrive/ColabNotebooks/group_3_ass_8_2024_2.html
